In [95]:
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt

In [96]:
from sklearn.cluster import KMeans
from sklearn.metrics import homogeneity_completeness_v_measure

In [97]:
from tfumap.paths import ensure_dir, MODEL_DIR, DATA_DIR

In [98]:
metrics_df = pd.concat([pd.read_pickle(i) for i in list((DATA_DIR / 'clustering_metric_df').glob('*.pickle'))])
metrics_df.loc[metrics_df.dataset == 'cassins', 'dataset'] = 'cassins_dtw'
metrics_df[:3]

,dataset,class_,dim,silhouette,homogeneity,completeness,v_measure,init_,n_clusters,model
0,mnist,PCA,2,0.377440,0.272552,0.39746,0.323363,0,5,"KMeans(n_clusters=5, random_state=0)"
1,mnist,PCA,2,0.377355,0.273045,0.39793,0.323865,1,5,"KMeans(n_clusters=5, random_state=1)"
2,mnist,PCA,2,0.377199,0.273357,0.39823,0.324184,2,5,"KMeans(n_clusters=5, random_state=2)"


In [99]:
metrics_df

,dataset,class_,dim,silhouette,homogeneity,completeness,v_measure,init_,n_clusters,model
0,mnist,PCA,2,0.377440,0.272552,0.397460,0.323363,0,5,"KMeans(n_clusters=5, random_state=0)"
1,mnist,PCA,2,0.377355,0.273045,0.397930,0.323865,1,5,"KMeans(n_clusters=5, random_state=1)"
2,mnist,PCA,2,0.377199,0.273357,0.398230,0.324184,2,5,"KMeans(n_clusters=5, random_state=2)"
3,mnist,PCA,2,0.377346,0.273049,0.397946,0.323874,3,5,"KMeans(n_clusters=5, random_state=3)"
4,mnist,PCA,2,0.377413,0.272501,0.397373,0.323298,4,5,"KMeans(n_clusters=5, random_state=4)"
...,...,...,...,...,...,...,...,...,...,...
45,fmnist,vae,2,0.375252,0.575557,0.511677,0.541740,0,14,"KMeans(n_clusters=14, random_state=0)"
46,fmnist,vae,2,0.374912,0.575514,0.511521,0.541634,1,14,"KMeans(n_clusters=14, random_state=1)"
47,fmnist,vae,2,0.374924,0.575444,0.511468,0.541573,2,14,"KMeans(n_clusters=14, random_state=2)"
48,fmnist,vae,2,0.374862,0.575051,0.511110,0.541198,3,14,"KMeans(n_clusters=14, random_state=3)"


In [100]:
metrics_df = metrics_df[
    ((metrics_df.dim == 64) &
    (metrics_df.class_ == 'scvis')) == False
]

In [101]:
metrics_df.class_.unique()

array(['PCA', 'ae_only', 'phate', 'autoencoder', 'scvis', 'direct', 'vae',
       'network', 'ivis', 'umap-learn', 'TSNE', 'parametric-tsne'],
      dtype=object)

### plot on the basis of the best silhouette score

In [102]:
metrics_df.dataset.unique()

array(['mnist', 'cassins_dtw', 'cifar10', 'macosko2015', 'fmnist'],
      dtype=object)

In [103]:
### get the best silhouette score for each dataset, class_, dim_
metrics_df_max_iter = []
for dataset in metrics_df.dataset.unique():
    for class_ in metrics_df.class_.unique():
        for dim in metrics_df.dim.unique():
            subset_df = metrics_df[
                (dataset == metrics_df.dataset.values) &
                (class_ == metrics_df.class_.values) &
                (dim == metrics_df.dim.values) 
            ]
            if len(subset_df) > 0:
                #print(np.argmax(subset_df.silhouette))
                metrics_df_max_iter.append(subset_df.iloc[np.argmax(subset_df.silhouette)])

In [104]:
metrics_max_df = pd.concat(metrics_df_max_iter, axis=1).T.reset_index()
metrics_max_df[:3]

,index,dataset,class_,dim,silhouette,homogeneity,completeness,v_measure,init_,n_clusters,model
0,0,mnist,PCA,2,0.377440,0.272552,0.397460,0.323363,0,5,"KMeans(n_clusters=5, random_state=0)"
1,18,mnist,PCA,64,0.091285,0.472775,0.532840,0.501013,3,8,KMeans(random_state=3)
2,5,mnist,ae_only,2,0.493923,0.307547,0.611768,0.409321,0,6,"KMeans(n_clusters=6, random_state=0)"


In [105]:
metrics_max_df['class_'].unique()

array(['PCA', 'ae_only', 'phate', 'autoencoder', 'scvis', 'direct', 'vae',
       'network', 'ivis', 'umap-learn', 'TSNE', 'parametric-tsne'],
      dtype=object)

In [106]:
metrics_max_df = metrics_max_df[["dataset","class_","dim","v_measure"]].set_index(['dataset', 'dim'])
metrics_max_df = metrics_max_df.pivot_table(
    index=["dataset", "dim"],
    columns="class_",
    values="v_measure",
    aggfunc="first",
)
metrics_max_df 

class_                PCA      TSNE   ae_only  autoencoder    direct  \
dataset     dim                                                        
cassins_dtw 2    0.602941  0.962837  0.598428     0.965995  0.953765   
            64   0.901887       NaN  0.750089     0.966182  0.959669   
cifar10     2    0.060510  0.068812  0.025795     0.072967  0.073806   
            64   0.059858       NaN  0.057365     0.074582  0.074152   
fmnist      2    0.422137  0.540777  0.481776     0.660207  0.649126   
            64   0.524356       NaN  0.554064     0.663545  0.660976   
macosko2015 2    0.552168  0.512415  0.569513     0.586217  0.556708   
            64   0.669553       NaN  0.589705     0.579288  0.782354   
mnist       2    0.323363  0.770399  0.409321     0.846037  0.851221   
            64   0.501013       NaN  0.425264     0.870142  0.774870   

class_               ivis   network  parametric-tsne     phate     scvis  \
dataset     dim                                                            
cassins_dtw 2    0.860182  0.968604         0.960494  0.727506  0.939898   
            64   0.904341  0.966486         0.943430  0.909418       NaN   
cifar10     2    0.069874  0.071882         0.038281  0.048486  0.064988   
            64   0.085892  0.074183         0.057020  0.048524       NaN   
fmnist      2    0.521658  0.659404         0.624835  0.626564  0.562736   
            64   0.561185  0.661821         0.468033  0.590483       NaN   
macosko2015 2    0.467066  0.711236         0.591243  0.558703  0.427783   
            64   0.495100  0.635610         0.468243  0.753938       NaN   
mnist       2    0.508190  0.782399         0.744601  0.672780  0.584140   
            64   0.532196  0.781833         0.497680  0.752232       NaN   

class_           umap-learn       vae  
dataset     dim                        
cassins_dtw 2      0.958064  0.683617  
            64     0.959621  0.792266  
cifar10     2      0.074285  0.055956  
            64     0.073321  0.090487  
fmnist      2      0.660286  0.531902  
            64     0.660182  0.563896  
macosko2015 2      0.551021  0.569113  
            64     0.776338  0.590808  
mnist       2      0.837543  0.546676  
            64     0.774738  0.561659

In [107]:
def can_float(x):
    try:
        float(x)
        if np.isnan(float(x)):
            return False
        return True
    except:
        return False

In [108]:
metric_string = (
    metrics_max_df[['TSNE', 'parametric-tsne', 'umap-learn',  'network', 'autoencoder',
        'scvis', 'ivis', 'phate', 'vae', 'ae_only', 'PCA']]
    .round(4)
    .to_latex()
    .replace("cassins\_dtw", "Cassin's")
    .replace("cifar10", "CIFAR10")
    .replace("fmnist", "FMNIST")
    .replace("mnist", "MNIST")
    .replace("macosko2015", "Retina")
    .replace("autoencoder", "UMAP/AE")
    .replace("ae\_only", "AE")
    .replace("network", "P. UMAP")
    .replace("umap-learn", "UMAP")
    .replace("vae", "VAE")
    .replace("pca", "PCA")
    .replace("parametric-tsne", "P. t-SNE")
    .replace("TSNE", "t-SNE")
    .replace("NaN", "-")
    .replace("scvis", "SCVIS")
    .replace("ivis", "IVIS")
    .replace("phate", "PHATE")
)

In [109]:
lines = metric_string.split('\n')
skip = 1
for line in lines:
    line_elements = line.split(' ')
    floatables = [can_float(le) for (le) in line_elements]
    floats = [float(j) for i, j in zip(floatables, line_elements) if i]
    if len(floats)> 1:
        best = np.argmax(floats[skip:])
        replace_element = np.where(floatables)[0][skip + best]
        line_elements[replace_element] = '\\textbf{' +line_elements[replace_element]+ '}'
    print(' '.join(line_elements))

\begin{tabular}{llrrrrrrrrrrr}
\toprule
      & class\_ &    t-SNE &  P. t-SNE &  UMAP &  P. UMAP &  UMAP/AE &   SCVIS &    IVIS &   PHATE &     VAE &  AE &     PCA \\
dataset & dim &         &                  &             &          &              &         &         &         &         &          &         \\
\midrule
Cassin's & 2  &  0.9628 &           0.9605 &      0.9581 &   \textbf{0.9686} &       0.9660 &  0.9399 &  0.8602 &  0.7275 &  0.6836 &   0.5984 &  0.6029 \\
      & 64 &     - &           0.9434 &      0.9596 &   \textbf{0.9665} &       0.9662 &     - &  0.9043 &  0.9094 &  0.7923 &   0.7501 &  0.9019 \\
CIFAR10 & 2  &  0.0688 &           0.0383 &      \textbf{0.0743} &   0.0719 &       0.0730 &  0.0650 &  0.0699 &  0.0485 &  0.0560 &   0.0258 &  0.0605 \\
      & 64 &     - &           0.0570 &      0.0733 &   0.0742 &       0.0746 &     - &  0.0859 &  0.0485 &  \textbf{0.0905} &   0.0574 &  0.0599 \\
FMNIST & 2  &  0.5408 &           0.6248 &      \textbf{0.6603} &  